In [20]:
import pandas as pd
import pandas_gbq

from github import Github
from google.cloud import bigquery
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [21]:
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [22]:
year = 'year.2017'
query = """SELECT created_at, repo.name FROM githubarchive.{0} WHERE RAND() < 5/(SELECT COUNT(*) FROM `githubarchive.{0}`) ORDER BY created_at;""".format(year) 
project_id = 'cs515-term'

In [23]:
import pandas as pd
df = pd.read_gbq(query, project_id=project_id, dialect='standard', credentials=credentials)

repos = df['name']
dates = df['created_at']

Downloading: 100%|██████████| 7/7 [00:00<00:00, 49.77rows/s]


In [24]:
from github import Github, UnknownObjectException, GithubException
from tqdm import tqdm

with open('password', 'r') as file:
    password = file.read().replace('\n', '')

g = Github("siddhik97", password)

language_array = []

for repo in tqdm(repos):
    try:
        repo_details = g.get_repo(repo)
        if repo_details.language is not None:
            language_array.append(repo_details.language)
        else:
            language_array.append("NaN")
    except (UnknownObjectException, GithubException):
        language_array.append("NaN")
        pass

RateLimitExceededException: 403 {"message": "API rate limit exceeded for user ID 60565142.", "documentation_url": "https://developer.github.com/v3/#rate-limiting"}

In [ ]:
import pandas as pd

df['lang'] = pd.DataFrame(language_array, columns=['languages'])
date_new = df["created_at"].astype(str).str.split(" ", n = 1, expand = True)
df["date"] = date_new[0]

final_df = pd.DataFrame(columns=['language', 'date'])
final_df['date'] = df['date']
final_df['language'] = df['lang']

lang_date_df = final_df[~final_df["language"].str.contains("NaN")]

In [ ]:
from matplotlib import pyplot as plt 
plt.figure(figsize=(20,10))
plt.suptitle('HISTOGRAM : Trending programming language for {0}'.format(year), fontsize=16)
plt.xlabel('programming languages', fontsize=14)
plt.ylabel('count', fontsize=14)
plt.hist(lang_date_df['language'])